In [16]:
#Imports and API Key

import pandas as pd
import quandl
from scipy import stats
import scipy
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt

%aimport dataclean

quandl.ApiConfig.api_key = 'm8FYMyoCaJSbTrBASNHh'

In [17]:
#pulling data from Quandl

data = pd.read_csv('data_for_pull.csv') #staging the QUANDL keys to pull in CSV
cols = list(data['Var_name'].astype('str'))
dataset = quandl.get([val for val in data['Quandl Key']]) #looping through the QUANDL keys to pull it into one DF
dataset.columns = cols

In [18]:
#pulling FED Yield Curve data

yields = pd.read_csv('Fed10Y_3M.csv')
yields['Date'] = pd.to_datetime(yields['Date'])
yields['Date'] = yields['Date'].apply(lambda x: x.strftime('%Y-%m'))
yields = yields.set_index('Date')
yields = yields.drop(['3 Month Treasury Yield', 'Rec_prob', 'NBER_Rec','Unnamed: 7'], axis=1) 

In [19]:
## back to working on the general data
dataset.index = dataset.index.strftime('%Y-%m') #converting the datetime index to Y/M so it is collapsable
dataset = dataset.groupby(dataset.index, as_index=True).agg(sum) #collapsing by Y/M

In [20]:
#converting GDP quarterly data into monthly

dataset = dataclean.convert_q_to_m(dataset, 'GDP')

#converting consumer sentiment into monthly

dataset = dataclean.convert_q_to_m(dataset, 'CONS_SENT')

In [21]:
#calculating change in GDP and converting Y into categorical values 
dataset['Recession'] = ((dataset['GDP'] - dataset['GDP'].shift(3)) < 0).astype(int)

In [22]:
#merge fed interest rate data here
dataset = dataset.join(yields, how='outer')

In [23]:
#cutoff most of missing data, Post March 2019, Prior 1959. CPI/PPI missing 2016 onward so need to cut that off
dataset = dataset.iloc[552:]
dataset = dataset.iloc[:-59]

In [24]:
y = dataset['Recession'] #splitting off Y

In [25]:
y_shift = y.shift(-3) #shifting y to forecast 3 months out

In [26]:
y_shift = y_shift.fillna(0)

In [27]:
dataset = dataset.drop(columns = ['GDP','Recession']) #dropping calc column and recession column from dataset, experimenting with taking out fed funds rate

In [28]:
#substituting mean value in for missing values and adding dummy column to indicate where done

for col in dataset.columns:
    dataclean.clean_zeros(col, dataset)

In [37]:
#adding momentum factors

momentum_cols = list(dataset.columns[:-6])

momentum_cols.remove('PPI') #removing PPI and CPI because they need a different transformation
momentum_cols.remove('CPI')

for i in [1,3,12]:
    for col in momentum_cols:
        dataclean.create_momentum(col,dataset,i)

In [38]:
#CPI Calcs

for i in [1,3,12]:
    for col in ['CPI','PPI']:
        dataclean.infl_momentum(col,dataset,i)

In [ ]:
### Data Prep Finished Here ###

In [40]:
dataset.columns

Index(['PMI', 'UNR', 'YUNR', 'CONS_SENT', 'HOME_SALES', 'PART_TIME',
       'CPPR_PRICE', 'HOUS_PERMS', 'HOUS_STARTS', 'CAP_UTIL', 'PERS_SAVINGS',
       'EXPORTS', 'IMPORTS', 'TRADE_BALANCE', 'INT_RATE', 'US_M2',
       'US_NHOME_SALES', 'PPI', 'CPI', '3YRT', '10 Year Treasury Yield',
       '3 Month Treasury Yield (Bond Equivalent Basis)', 'Spread',
       'HOME_SALES_PXY', 'CPPR_PRICE_PXY', 'HOUS_PERMS_PXY', 'CAP_UTIL_PXY',
       'US_NHOME_SALES_PXY', '3YRT_PXY', 'PMI_1m_shift', 'UNR_1m_shift',
       'YUNR_1m_shift', 'CONS_SENT_1m_shift', 'HOME_SALES_1m_shift',
       'PART_TIME_1m_shift', 'CPPR_PRICE_1m_shift', 'HOUS_PERMS_1m_shift',
       'HOUS_STARTS_1m_shift', 'CAP_UTIL_1m_shift', 'PERS_SAVINGS_1m_shift',
       'EXPORTS_1m_shift', 'IMPORTS_1m_shift', 'TRADE_BALANCE_1m_shift',
       'INT_RATE_1m_shift', 'US_M2_1m_shift', 'US_NHOME_SALES_1m_shift',
       '3YRT_1m_shift', '10 Year Treasury Yield_1m_shift',
       '3 Month Treasury Yield (Bond Equivalent Basis)_1m_shift',
    